In [ ]:
import torch
import torchaudio
import soundfile as sf
import matplotlib.pyplot as plt
import librosa
import os
from audio_dataset import audio_segment
import numpy as np

working_dir = os.path.join(os.getcwd().split('Text2BGAudio')[0],'Text2BGAudio')
os.chdir(working_dir)
print(f"{working_dir=}")

working_dir='c:\\Users\\amitmils\\Documents\\Repo\\Text2BGAudio'


### Music Dataset

In [10]:
music_data_raw_folder = r"_Data\Music\train"
save_folder = r"_Data/Music"
SR = 8000

In [9]:
data_set = []
for label in os.listdir(music_data_raw_folder):
    curr_size = len(data_set)
    title = f"Processing label: {label}"
    print("#" * len(title))
    print(title)
    label_folder_path = os.path.join(music_data_raw_folder, label)
    for audio_file in os.listdir(label_folder_path):
        audio_file_path = os.path.join(label_folder_path, audio_file)
        waveform, orig_sample_rate = librosa.load(audio_file_path)
        waveform = torch.tensor(waveform)
        if SR != orig_sample_rate:
            waveform = torchaudio.transforms.Resample(orig_sample_rate, SR)(waveform)
        segment_length = 10 * SR
        batches = waveform.split(segment_length)
        for batch_seg in batches:
            if len(batch_seg) < segment_length:
                continue
            data_set.append((batch_seg.numpy().astype(np.float32),SR,label,audio_file_path))
    print(f"Created {len(data_set) - curr_size} segments")
    print("#" * len(title)  + "\n\n")

torch.save(data_set, os.path.join(save_folder, f"music_dataset_{os.path.basename(music_data_raw_folder)}_size{len(data_set)}.pt"))

#######################
Processing label: Angry
Created 10 segments
#######################


#####################
Processing label: Joy
Created 21 segments
#####################


######################
Processing label: Love
Created 10 segments
######################


#####################
Processing label: Sad
Created 51 segments
#####################


#######################
Processing label: Scary
Created 26 segments
#######################


##########################
Processing label: Surprise
Created 10 segments
##########################




In [14]:
data_set = []
output_folder = r"_Data/Music/train_segments"
for label in os.listdir(music_data_raw_folder):
    num_segments = 0
    title = f"Processing label: {label}"
    print("#" * len(title))
    print(title)
    label_folder_path = os.path.join(music_data_raw_folder, label)
    output_label_folder = os.path.join(output_folder, label)
    for audio_file in os.listdir(label_folder_path):
        audio_file_path = os.path.join(label_folder_path, audio_file)
        waveform, orig_sample_rate = librosa.load(audio_file_path)
        waveform = torch.tensor(waveform)
        if SR != orig_sample_rate:
            waveform = torchaudio.transforms.Resample(orig_sample_rate, SR)(waveform)
        segment_length = 10 * SR
        batches = waveform.split(segment_length)

        for i,batch_seg in enumerate(batches):
            if len(batch_seg) < segment_length:
                continue
            segment_name = os.path.join(output_label_folder,f"{audio_file.split('.')[0]}_{i}.wav")
            os.makedirs(output_label_folder, exist_ok=True)
            sf.write(segment_name, batch_seg.numpy().astype(np.float32), SR)
        num_segments+=i+1
    print(f"Created {num_segments} segments")
    print("#" * len(title)  + "\n\n")


#######################
Processing label: Angry
Created 1131 segments
#######################


#####################
Processing label: Joy
Created 1662 segments
#####################


######################
Processing label: Love
Created 1939 segments
######################


#####################
Processing label: Sad
Created 1233 segments
#####################


#######################
Processing label: Scary
Created 1231 segments
#######################


##########################
Processing label: Surprise
Created 360 segments
##########################




In [16]:
import pandas as pd
import os
import random


In [23]:
data_tsv = list()
for root, dirs, files in os.walk(r"C:\Users\amitmils\Documents\Repo\Text2BGAudio\_Data\Music\train_segments"):
    for file in files:
        data_tsv.append(
            {
                "name": file.split(".")[0],
                "dataset": "MoodAudio",
                "caption": os.path.basename(root).lower(),
                "audio_path": os.path.join("/content/drive/MyDrive/Colab Notebooks/AmitM/Make-An-Audio/data",root.split('Music\\')[-1], file).replace('\\','/'),
            }
        )
random.shuffle(data_tsv)
df = pd.DataFrame(data_tsv)
df.to_csv(r"C:\Users\amitmils\Documents\Repo\Text2BGAudio\_Data\Music\train_segments.tsv", sep="\t", index=False)